In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import financial.data as fd
import financial.model as fm
import financial.portfolio as fp
import financial.strategies.simulation as fss
from financial.io.file.cache import FileCache
from financial.io.cache import AppendStrategy
from financial.io.file.model import FileModelProvider

import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory

2025-07-20 10:24:44.539125: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-20 10:24:44.794476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752999884.941691    8662 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752999884.986486    8662 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752999885.254197    8662 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
load_dotenv(dotenv_path=find_dotenv())

cache = os.environ["CACHE"] + "/"
model = os.environ["MODEL"]

ds = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache))
mp = FileModelProvider(model_path=model)
mc = fm.ModelCache(ds, mp, cache=FileCache(update_strategy=AppendStrategy(), cache_path=model))
uc = fm.UnifiedCache(ds, mc)

In [5]:
from financial.momentum.indicators.modelIndicator import ModelIndicator
simulation_indicator = ModelIndicator("rnn")

In [6]:
simulation_universe = ['BLK', '^IBEX', 'NVDA', 'BAM', 'GOOG', 'MSTF', 'AAPL']
import financial.strategies.allocation as fsa
import financial.strategies.filter as fsf
import financial.strategies.rank as fsr

simulation_filter = fsf.CompositeAssetFilter( [fsf.TopKAssetFilter(k=4), fsf.MinimumValueAssetFilter(threshold=0.0)] )
simulation_allocation = fsa.EqualWeightAllocation()
simulation_strategy = fsr.AssetRankingStrategy("Simulation", simulation_universe, simulation_indicator, simulation_filter, simulation_allocation)
print(simulation_strategy)

Simulation
- 7 asset universe: ['BLK', '^IBEX', 'NVDA', 'BAM', 'GOOG', 'MSTF', 'AAPL']
- Indicator: {'type': 'financial.momentum.indicators.modelIndicator.ModelIndicator'}


In [7]:
%%time
import financial.strategies.rebalance as fsrebalance

strategy_simulation = "simulation"
strategy = simulation_strategy
benchmark = fp.BenchmarkPortfolio("^GSPC")
market = ds.get_data("^GSPC")
rebalancing = fsrebalance.MonthlyRebalancingSchedule(-1,market) # +1 first day, -1 last day, +2 second day with 1st day momentum

print("STRATEGY")
print(strategy)
print("BENCHMARK")
print(benchmark)
print("REBALANCING")
print(rebalancing)

simulation = fss.StockStrategySimulation(uc, strategy, rebalancing, market)
simulation.verbose = True  # verbose 41.7s vs. 27s without trading details
simulation_start_year = 2010
simulation_end_year = 2024
simulation.simulate(start_year=simulation_start_year, end_year=simulation_end_year)

STRATEGY
Simulation
- 7 asset universe: ['BLK', '^IBEX', 'NVDA', 'BAM', 'GOOG', 'MSTF', 'AAPL']
- Indicator: {'type': 'financial.momentum.indicators.modelIndicator.ModelIndicator'}
BENCHMARK
^GSPC(^GSPC)
REBALANCING
Rebalancing schedule: Monthly rebalancing on -1


KeyError: 'rnn_BLK_2025_single'

In [8]:
year_returns = simulation.year_returns()

print("\nSTRATEGY RETURNS")
for year in year_returns:
    print(f"{year}: {100*year_returns[year]:6.2f}%")


cumulative = simulation.cumulative_returns(year_returns)

print("\nCUMULATIVE RETURNS")
for year in cumulative:
    print(f"{year}: {100*cumulative[year]:6.2f}%")import financial.portfolios.statistics as fps

SyntaxError: invalid syntax (4157515791.py, line 12)

In [ ]:
assets = {'^GSPC': 1.0}
target = fp.WeightedPortfolio.from_assets("Target", assets)
benchmark_returns = target.returns(ds,'2010-12-31','2024-12-31')
benchmark_monthly_returns = fss.SimulationUtilities.monthly_returns(ds,target,2010,2024)
benchmark_cumulative_returns = fps.CumulativeReturn().get_series(benchmark_monthly_returns)

In [ ]:
benchmark_monthly_returns = fss.SimulationUtilities.monthly_returns(ds,target,simulation_start_year,simulation_end_year)
benchmark_cumulative_returns = fps.CumulativeReturn().get_series(benchmark_monthly_returns)
fss.SimulationReport.simulation_report(benchmark_monthly_returns, fss.accumulate_return)

In [ ]:
rotation = simulation.monthly_rotation()
print(f"Average monthly rotation: {100*rotation:.2f}%")

In [ ]:
year_returns = simulation.year_returns()
#print(year_returns)

print("\nSTRATEGY RETURNS")
for year in year_returns:
    print(f"{year}: {100*year_returns[year]:6.2f}%")


cumulative = simulation.cumulative_returns(year_returns)

print("\nCUMULATIVE RETURNS")
for year in cumulative:
    print(f"{year}: {100*cumulative[year]:6.2f}%")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl

mpl.rc('figure', figsize=(16, 9))
mpl.__version__

# Adjusting the style of matplotlib
style.use('ggplot')

fig = plt.figure()

benchmark = 100*benchmark_cumulative_returns
#benchmark = 100*fps.MaximumDrawdown().get_series(benchmark_monthly_returns)

data = 100*cumulative_returns
#data = 100*drawdown

#monthly_returns.plot(label=ticker)
data.plot()
benchmark.plot()